In [ ]:
import os

import cartopy
import cartopy.crs as ccrs
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas
import rasterio

from ghaa.config import load_config
from ghaa.plot.map import get_axes, plot_basemap, plot_basemap_labels, scale_bar

In [ ]:
base_path = load_config()["base_path"]
base_path

In [ ]:
color_codes = {
    "ocean": "#bfc0bf",
    "land": "#e4e4e3",
    "flash flooding moderate scenario": "#a2cfe3",
    "flash flooding high scenario": "#0088b7",
    "landslide": "#d5c68e",
    "drought": "",
    "healthcare": "#9f71a4",
    "exposure": "#d53c17",
    "buildings": "#e3e2de",
    "electricity": "#0f6cb2",
    "water": "#87cefa",
    "transport": "#ffd700"
}

In [ ]:
ax = get_axes()
plot_basemap(ax, os.path.join(base_path, 'data'), plot_regions=True)
plot_basemap_labels(ax, os.path.join(base_path, 'data'), include_regions=True)
scale_bar(ax)

ax

## Raster plot from access CSV

In [ ]:
pop_path = os.path.join(
    base_path, 'incoming', 'population', 'Population Density& Demographics', 
    'ID 23_Population_Density_Demographics - fb AI', 'population_gha_2019-07-01_geotiff', 
    'population_gha_2019-07-01.tif')

In [ ]:
access_path = os.path.join(
    base_path, 'results', 'proximity_results', 'population_gha_2019-07-01_proximity_distance.csv')

In [ ]:
access_tif_path = os.path.join(
    base_path, 'results', 'proximity_results', 'population_gha_2019-07-01_proximity_distance.tif')

In [ ]:
with rasterio.open(pop_path) as ds:
    pass

In [ ]:
Z = np.full(ds.shape, -999.0)

In [ ]:
access = pandas.read_csv(access_path, usecols=['length_km', 'access', 'x', 'y'])
access.head()

In [ ]:
for obs in access.itertuples():
    row, col = ds.index(obs.x, obs.y)
    Z[row, col] = obs.length_km

In [ ]:
with rasterio.open(
        access_tif_path,
        'w',
        driver='GTiff',
        height=Z.shape[0],
        width=Z.shape[1],
        count=1,
        dtype=Z.dtype,
        crs=ds.crs,
        transform=ds.transform,
        nodata=-999.0
    ) as new_dataset:
    new_dataset.write(Z, 1)